# Nerfstudio Training: NeRF vs 3D Gaussian Splatting

This notebook trains both methods using Nerfstudio for fair comparison.

**Methods:**
- NeRF: `nerfacto` (fast NeRF variant)
- 3DGS: `splatfacto` (Gaussian Splatting with gsplat)

**Expected Time:** ~5-10 minutes each

## Step 1: Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Install Nerfstudio

In [ ]:
# Install nerfstudio and dependencies
!pip install nerfstudio -q

print("\n✓ Nerfstudio installed")

# Verify installation
!ns-train --help | head -20

## Step 3: Setup Working Directory

In [ ]:
import os

# Create working directory
%cd /content
!mkdir -p nerfstudio_training
%cd nerfstudio_training

print("\n✓ Working directory created")
!pwd

## Step 4: Mount Drive & Copy Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✓ Drive mounted")

In [ ]:
# Copy dataset
!mkdir -p data/nerf_synthetic
!cp -r /content/drive/MyDrive/data/nerf_synthetic/lego data/nerf_synthetic/

print("✓ Dataset copied")

# Verify
!ls -lh data/nerf_synthetic/lego/

## Step 5: Convert Dataset for Nerfstudio

Nerfstudio needs the data in a specific format

In [ ]:
# Nerfstudio can read blender format directly!
# Just need to verify the structure

import json

print("Checking dataset format...\n")

for split in ['train', 'val', 'test']:
    json_path = f'data/nerf_synthetic/lego/transforms_{split}.json'
    if os.path.exists(json_path):
        with open(json_path) as f:
            data = json.load(f)
        print(f"✓ {split}: {len(data['frames'])} frames")
    else:
        print(f"✗ {split}: not found")

print("\n✓ Dataset ready for Nerfstudio")

## Step 6: Train NeRF (Nerfacto)

**This will take ~5-10 minutes**

In [ ]:
# Train NeRF using nerfacto method
!ns-train nerfacto \
    --data data/nerf_synthetic/lego \
    --pipeline.datamanager.camera-optimizer.mode off \
    --max-num-iterations 10000 \
    --viewer.quit-on-train-completion True \
    blender-data

## Step 7: Evaluate NeRF Results

In [ ]:
# Find the latest nerfacto config
!ls -lht outputs/lego/nerfacto/*/config.yml | head -1

# Store the path (you'll need to update this with actual path)
nerf_config = !ls outputs/lego/nerfacto/*/config.yml | head -1
nerf_config = nerf_config[0]
print(f"\nNeRF config: {nerf_config}")

In [ ]:
# Evaluate on test set
!ns-eval \
    --load-config {nerf_config} \
    --output-path nerf_eval_results.json

In [ ]:
# View results
import json

with open('nerf_eval_results.json', 'r') as f:
    nerf_results = json.load(f)

print("="*60)
print("NERF (NERFACTO) RESULTS")
print("="*60)
print(json.dumps(nerf_results, indent=2))
print("="*60)

## Step 8: Train 3D Gaussian Splatting (Splatfacto)

**This will take ~5-10 minutes**

In [ ]:
# Train 3DGS using splatfacto method
!ns-train splatfacto \
    --data data/nerf_synthetic/lego \
    --pipeline.datamanager.camera-optimizer.mode off \
    --max-num-iterations 10000 \
    --viewer.quit-on-train-completion True \
    blender-data

## Step 9: Evaluate 3DGS Results

In [ ]:
# Find the latest splatfacto config
!ls -lht outputs/lego/splatfacto/*/config.yml | head -1

# Store the path
gs_config = !ls outputs/lego/splatfacto/*/config.yml | head -1
gs_config = gs_config[0]
print(f"\n3DGS config: {gs_config}")

In [ ]:
# Evaluate on test set
!ns-eval \
    --load-config {gs_config} \
    --output-path gs_eval_results.json

In [ ]:
# View results
with open('gs_eval_results.json', 'r') as f:
    gs_results = json.load(f)

print("="*60)
print("3D GAUSSIAN SPLATTING (SPLATFACTO) RESULTS")
print("="*60)
print(json.dumps(gs_results, indent=2))
print("="*60)

## Step 10: Side-by-Side Comparison

In [ ]:
import pandas as pd

# Create comparison table
comparison = {
    'Metric': [],
    'NeRF (Nerfacto)': [],
    '3DGS (Splatfacto)': []
}

# Extract metrics
for key in ['psnr', 'ssim', 'lpips']:
    if key in nerf_results['results']:
        comparison['Metric'].append(key.upper())
        comparison['NeRF (Nerfacto)'].append(f"{nerf_results['results'][key]:.4f}")
        comparison['3DGS (Splatfacto)'].append(f"{gs_results['results'][key]:.4f}")

df = pd.DataFrame(comparison)

print("\n" + "="*60)
print("COMPARISON: NeRF vs 3D Gaussian Splatting")
print("="*60)
print(df.to_string(index=False))
print("="*60)

# Save to CSV
df.to_csv('comparison_results.csv', index=False)
print("\n✓ Saved to comparison_results.csv")

## Step 11: Render Test Images

In [ ]:
# Render NeRF test images
!ns-render camera-path \
    --load-config {nerf_config} \
    --camera-path-filename data/nerf_synthetic/lego/transforms_test.json \
    --output-path renders/nerf/

print("\n✓ NeRF images rendered to renders/nerf/")

In [ ]:
# Render 3DGS test images
!ns-render camera-path \
    --load-config {gs_config} \
    --camera-path-filename data/nerf_synthetic/lego/transforms_test.json \
    --output-path renders/3dgs/

print("\n✓ 3DGS images rendered to renders/3dgs/")

## Step 12: View Sample Images

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Show first rendered image from each method
nerf_imgs = !ls renders/nerf/*.png | head -3
gs_imgs = !ls renders/3dgs/*.png | head -3

print("Sample Renderings:\n")

for i in range(min(3, len(nerf_imgs))):
    print(f"\n--- Image {i+1} ---")
    print("NeRF:")
    display(Image(nerf_imgs[i], width=300))
    print("3DGS:")
    display(Image(gs_imgs[i], width=300))

## Step 13: Download Results

In [ ]:
from google.colab import files

# Zip everything
!zip -r nerfstudio_results.zip \
    nerf_eval_results.json \
    gs_eval_results.json \
    comparison_results.csv \
    renders/

print("\n✓ Results zipped")

# Download
files.download('nerfstudio_results.zip')

print("\n✓ Download started")

## Step 14: Summary for Report

In [ ]:
print("="*60)
print("SUMMARY FOR YOUR REPORT")
print("="*60)
print(f"\nNeRF (Nerfacto):")
print(f"  PSNR: {nerf_results['results']['psnr']:.2f} dB")
print(f"  SSIM: {nerf_results['results']['ssim']:.4f}")
print(f"  LPIPS: {nerf_results['results']['lpips']:.4f}")

print(f"\n3D Gaussian Splatting (Splatfacto):")
print(f"  PSNR: {gs_results['results']['psnr']:.2f} dB")
print(f"  SSIM: {gs_results['results']['ssim']:.4f}")
print(f"  LPIPS: {gs_results['results']['lpips']:.4f}")

# Calculate differences
psnr_diff = gs_results['results']['psnr'] - nerf_results['results']['psnr']
ssim_diff = gs_results['results']['ssim'] - nerf_results['results']['ssim']

print(f"\nDifference (3DGS - NeRF):")
print(f"  PSNR: {psnr_diff:+.2f} dB ({'+' if psnr_diff > 0 else ''}{(psnr_diff/nerf_results['results']['psnr']*100):.1f}%)")
print(f"  SSIM: {ssim_diff:+.4f} ({'+' if ssim_diff > 0 else ''}{(ssim_diff/nerf_results['results']['ssim']*100):.1f}%)")

print("\n" + "="*60)
print("\nBoth methods trained for 10,000 iterations using Nerfstudio.")
print("Dataset: NeRF Synthetic LEGO (Blender)")
print("="*60)

---

## Notes:

**Advantages of this approach:**
- ✅ Same framework (Nerfstudio) for both methods
- ✅ Fair comparison with equal iterations
- ✅ Much faster than vanilla implementations
- ✅ Automatic metrics calculation
- ✅ Professional rendered outputs

**For your report:**
- Both trained for 10k iterations
- Same dataset and hardware
- Apples-to-apples comparison
- Can cite Nerfstudio framework